In [1]:
import sys
sys.path.append("../")
from plotting import CandlePlot
import pandas as pd

In [2]:
from technicals.patterns import apply_patterns

In [3]:
df = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [4]:
df_an = df[['time', 'mid_o', 'mid_h', 'mid_l','mid_c']].copy()

In [6]:
df_an = apply_patterns(df_an)

TypeError: object of type 'NoneType' has no len()

In [ ]:
df_an.tail()

In [ ]:
ENGULFING_FACTOR = 1.1
TWEEZER_BODY = 15.0
TWEEZER_HL = 0.01
TWEEZER_TOP_BODY = 40.0
TWEEZER_BOTTOM_BODY = 60.0
MORNING_STAR_PREV2_BODY = 90.0
MORNING_STAR_PREV_BODY = 10.0

apply_marubozu = lambda x: x.body_perc > 98.0

def apply_engulfing(row):
    if row.direction != row.direction_prev:
        if row.body_size > row.body_size_prev * ENGULFING_FACTOR:
            return True
    return False

def apply_tweezer_top(row):
    if abs(row.body_size_change) < TWEEZER_BODY:
        if row.direction == -1 and row.direction != row.direction_prev:
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
                if row.body_top_perc < TWEEZER_TOP_BODY:
                    return True
    return False

def apply_tweezer_bottom(row):
    if abs(row.body_size_change) < TWEEZER_BODY:
        if row.direction == 1 and row.direction != row.direction_prev:
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
                if row.body_bottom_perc > TWEEZER_BOTTOM_BODY:
                    return True
    return False

def apply_morning_star(row, direction=1):
    if row.body_perc_prev_2 > MORNING_STAR_PREV2_BODY:
        if row.body_perc_prev < MORNING_STAR_PREV_BODY:
            if row.direction == direction and row.direction_prev_2 != direction:
                if direction == 1:
                    if row.mid_c > row.mid_point_prev_2:
                        return True
                else:
                    if row.mid_c < row.mid_point_prev_2:
                        return True
    return False
                    

In [ ]:
df_an['body_size_prev'] = df_an.body_size.shift(1)
df_an['direction_prev'] = df_an.direction.shift(1)
df_an['direction_prev_2'] = df_an.direction.shift(2)
df_an['body_perc_prev'] = df_an.body_perc.shift(1)
df_an['body_perc_prev_2'] = df_an.body_perc.shift(2)

df_an['HANGING_MAN'] = [True if df_an.body_bottom_perc[i] > 75.0 and df_an.body_perc[i] < 15.0 else False for i in range(len(df_an))]
df_an['SHOOTING_STAR'] = [True if df_an.body_top_perc[i] < 25.0 and df_an.body_perc[i] < 15.0 else False for i in range(len(df_an))]
df_an['SPINNING_TOP'] = [True if df_an.body_top_perc[i] < 60.0 and df_an.body_bottom_perc[i] > 40.0 and df_an.body_perc[i] < 15.0 else False for i in range(len(df_an))]
df_an['MARUBOZU'] = df_an.apply(apply_marubozu, axis=1)
df_an['ENGULFING'] = df_an.apply(apply_engulfing, axis=1)
df_an['TWEEZER_TOP'] = df_an.apply(apply_tweezer_top, axis=1)
df_an['TWEEZER_BOTTOM'] = df_an.apply(apply_tweezer_bottom, axis=1)
df_an['MORNING_STAR'] = df_an.apply(apply_morning_star, axis=1)
df_an['EVENING_STAR'] = df_an.apply(apply_morning_star, axis=1, direction=-1)

In [ ]:
df_an[df_an['EVENING_STAR'] == True]

In [ ]:
import plotly.graph_objects as go

MARKER = '#0066FF'
dfp = df_an.iloc[26000:28000]
cp = CandlePlot(dfp, candles=True)
df_temp = cp.df_plot[cp.df_plot.EVENING_STAR == True]
cp.fig.add_trace(go.Candlestick(
          x = df_temp.sTime,
          open = df_temp.mid_o,
          high = df_temp.mid_h,
          low = df_temp.mid_l,
          close = df_temp.mid_c,
          line=dict(width=1), 
          increasing_fillcolor = MARKER,
          decreasing_fillcolor = MARKER,
          increasing_line_color= MARKER,
          decreasing_line_color = MARKER,
          opacity=1,
          
      ))
cp.show_plot()